# Regresión OLS Práctica
Katlyn Goeujon-Mackness <br>
31/31/2025 <br>
**Descripción:** Un espacio para practicar el método OLS de regresión lineal.

## Estructura
### Intervalos de confianza en regresión
* Descubrir cómo calcular pronósticos puntuales y sus intervalos de confianza en el contexto de la regresión lineal múltiple. 
* Utilizar la distribución T de Student y la desviación estándar de la regresión para evaluar la incertidumbre en las predicciones del modelo.

### Análisis de series de tiempo
* Identificar patrones y tendencias en datos temporales.
* Explorar técnicas para modelar y predecir comportamientos futuros basados en datos históricos.
